<center> <font color = #82318E size = 5 face = "微软雅黑" > RBF Tensor einsum Version</font> </center>

<center> <font color = #82318E size = 4 face = "微软雅黑" > 1. Prepare Data </font> </center>

In [2]:
import numpy as np
np.random.seed(522)

im = 9

temp = np.random.normal(loc = 0, scale = 1, size = (im, 13))

H = temp[::, 0:3:1]
Tx = temp[::, 4:7:1]
Ty = temp[::, 7:10:1]
Tz = temp[::, -3::1]

print(' The shape of H  is {};\n The shape of Tx is {};\n The shape of Ty is {};\n The shape of Tz is {}.'.format(
      H.shape,Tx.shape,Ty.shape,Tz.shape))

T = np.concatenate((Tx, Ty, Tz), axis = 0).reshape(-1, im, 3)
print(T.shape)

 The shape of H  is (9, 3);
 The shape of Tx is (9, 3);
 The shape of Ty is (9, 3);
 The shape of Tz is (9, 3).
(3, 9, 3)


<center> <font color = #82318E size = 4 face = "微软雅黑" > 2. Original Version </font> </center>

<table border = '0' witdth = '2000' >
<tr> 
<td width = '2000' ><img src = ./TensorOrig.png /></td> 
</tr>
</table>

In [3]:
p = np.multiply(H[:,0], Tx[:,0]) + np.multiply(H[:,1], Ty[:,0]) + np.multiply(H[:,2], Tz[:,0])
q = np.multiply(H[:,0], Tx[:,1]) + np.multiply(H[:,1], Ty[:,1]) + np.multiply(H[:,2], Tz[:,1])
s = np.multiply(H[:,0], Tx[:,2]) + np.multiply(H[:,1], Ty[:,2]) + np.multiply(H[:,2], Tz[:,2])
print(p.shape, q.shape, s.shape)

AIM = np.concatenate((p.reshape(-1, 1), q.reshape(-1, 1), s.reshape(-1, 1)), axis = 1)
print(AIM)

(9,) (9,) (9,)
[[-0.4702549  -1.09373498  0.71895093]
 [-4.34420159  1.93786275  1.27702953]
 [-0.79210234 -0.6952026  -0.78218299]
 [ 1.90391861 -1.60849709 -2.27491605]
 [ 4.405149    0.28022822 -1.9293554 ]
 [ 1.17188343 -1.65311901 -0.04564163]
 [-1.65806898 -0.72016552 -0.64827817]
 [ 2.80186909  0.9698682  -0.11173106]
 [-3.2307313  -2.67230977 -1.38377616]]


<center> <font color = #82318E size = 4 face = "微软雅黑" > 2. Tensor einsum Version </font> </center>

In [7]:
R = np.einsum('ik,kij->ij', H, T)
print("R=\n",R)

R=
 [[-0.4702549  -1.09373498  0.71895093]
 [-4.34420159  1.93786275  1.27702953]
 [-0.79210234 -0.6952026  -0.78218299]
 [ 1.90391861 -1.60849709 -2.27491605]
 [ 4.405149    0.28022822 -1.9293554 ]
 [ 1.17188343 -1.65311901 -0.04564163]
 [-1.65806898 -0.72016552 -0.64827817]
 [ 2.80186909  0.9698682  -0.11173106]
 [-3.2307313  -2.67230977 -1.38377616]]


<H1>注释：</H1>
<pre><code>
原始公式：
p = H(:,1)*Tx(:,1) + H(:,2)*Ty(:,1) + H(:,3)*Tz(:,1) 
q = H(:,1)*Tx(:,2) + H(:,2)*Ty(:,2) + H(:,3)*Tz(:,2)
s = H(:,1)*Tx(:,3) + H(:,2)*Ty(:,3) + H(:,3)*Tz(:,3) 

猜想如果用爱因斯坦求和约定来表示可能更简单：

先设R=[p q s], H=[u v w], T = np.stack((Tx, Ty, Tz), axis = 0), 上面的表达式可以改写为：
R[:,1] = H(:,1)*T(1,:,1) + H(:,2)*T(2,:,1) + H(:,3)*T(3,:,1) 
R[:,2] = H(:,1)*T(1,:,2) + H(:,2)*T(2,:,2) + H(:,3)*T(3,:,2)
R[:,3] = H(:,1)*T(1,:,3) + H(:,2)*T(2,:,3) + H(:,3)*T(3,:,3) 

写出循环形式：
for i in range(9):
    for j in range(3):
        for k in range(3):
        	R[i, j] += H[i, k] .* T[k, i, j]


变换一下，去掉k循环：
for i in range(9):
    for j in range(3):
        	R[i, j] = sum_k H[i, k] .* T[k, i, j] 

写出爱因斯坦求和约定形式：
einsum('ik,kij->ij', H, T)	
一行搞定！
</pre></code>

<H1>备注：TensorFlow如何实现张量收缩</H1>
<H5>https://www.w3cschool.cn/tensorflow_python/tensorflow_python-qszk2e44.html</H5>
<pre><code>一般来说, 方程是从较熟悉的元素方程得到：
删除变量名称、括号和逗号；
    1）用 "*" 替换 "，"；
    2）删除总和标志；
    3)将输出移到右侧，并将 "=" 替换为 "->>"。
    4)许多常见操作可以用这种方式来表示。
    
    例如:
    # Matrix multiplication
    >>> einsum('ij,jk->ik', m0, m1)  # output[i,k] = sum_j m0[i,j] * m1[j, k]

    # Dot product
    >>> einsum('i,i->', u, v)  # output = sum_i u[i]*v[i]

    # Outer product
    >>> einsum('i,j->ij', u, v)  # output[i,j] = u[i]*v[j]

    # Transpose
    >>> einsum('ij->ji', m)  # output[j,i] = m[i,j]

    # Batch matrix multiplication
    >>> einsum('aij,ajk->aik', s, t)  # out[a,i,k] = sum_j s[a,i,j] * t[a, j, k]  
</pre></code>